In [13]:
import boto3
import json
import cv2
import numpy as np

In [14]:
client = boto3.client('sagemaker-runtime')

In [22]:
def image_file_to_tensor(path):
    """Reads an image file and coverts it to a tensor (ndarray). 
    
    No resizing or cropping is done, so the image dimensions must match
    the model input shape (224x224 for the resnet model).
    
    Args: 
        path (str): The file name or path to the image file.
    """
    
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = np.asarray(image)
    image = cv2.normalize(image.astype('float'), None, 0, 1, cv2.NORM_MINMAX)
    image = np.expand_dims(image, axis=0)
    return image

def add_imagenet_labels(prediction_result):
    """Add imagenet class labels to the prediction result. The
    prediction_result argument will be modified in place.
    """

    # read the labels from a file
    labels = []
    with open('labels.txt', 'r') as f:
        labels = [l.strip() for l in f]
    
    # add labels to the result dict
    for pred in prediction_result['predictions']:
        prediction_labels = [labels[x - 1] for x in pred['classes']]
        pred['labels'] = prediction_labels

        
def print_probabilities_and_labels(labelled_result):
    """Print the labelled results."
    """
        
    for pred in labelled_result['predictions']:
        for i in range(0, len(pred['labels'])):
            print('{:1.7f} {}'.format(
                pred['probabilities'][i],
                pred['labels'][i],
            ))
        print()

In [23]:
endpoint_name = 'tensorflow-inference-2020-06-19-15-40-58-015'
kitten_image = image_file_to_tensor('kitten.jpg')
payload = json.dumps(kitten_image.tolist())

In [24]:
payload = json.dumps(kitten_image.tolist())

In [25]:
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType='application/json',
)

result = json.loads(response['Body'].read().decode())
add_imagenet_labels(result)
print_probabilities_and_labels(result)

0.5246838 n02123045 tabby, tabby cat
0.4617371 n02123159 tiger cat
0.0133929 n02124075 Egyptian cat

